In [2]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [1]:
import yaml

config_path = './config_files/baseline_small_bs1024.yaml'

with open(config_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)

In [2]:

import os
os.environ['HF_HOME'] = config['dataset']['hf_home']
os.environ['HF_DATASETS_CACHE'] = config['dataset']['hf_cache']

os.environ['XLA_FLAGS'] ='--xla_gpu_cuda_data_dir=/usr/lib/cuda'

In [3]:
import tensorflow as tf
print(tf.__version__)

2024-05-31 14:55:12.802275: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 14:55:12.802310: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 14:55:12.803951: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-31 14:55:12.812529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 14:55:14.157853: W tensorflow/compiler/tf2

2.15.1


In [4]:
from dlomix.data import load_processed_dataset
dataset = load_processed_dataset(config['dataset']['processed_path'])

/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [5]:
for x in dataset.tensor_val_data:
    print(x)
    break

2024-05-31 14:55:16.666191: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


({'modified_sequence': <tf.Tensor: shape=(1024, 30), dtype=int64, numpy=
array([[21,  6, 14, ...,  0,  0,  0],
       [21,  6,  5, ...,  0,  0,  0],
       [21, 18,  7, ...,  0,  0,  0],
       ...,
       [21,  6, 13, ...,  0,  0,  0],
       [21, 17, 12, ...,  0,  0,  0],
       [21,  8, 13, ...,  0,  0,  0]])>, 'precursor_charge_onehot': <tf.Tensor: shape=(1024, 6), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)>, 'collision_energy_aligned_normed': <tf.Tensor: shape=(1024,), dtype=float32, numpy=
array([0.21443671, 0.3644403 , 0.35      , ..., 0.25060007, 0.21204613,
       0.35      ], dtype=float32)>, 'method_nbr': <tf.Tensor: shape=(1024,), dtype=float32, numpy=array([2., 2., 1., ..., 2., 2., 1.], dtype=float32)>}, <tf.Tensor: shape=(1024, 174), dtype=float32, numpy=
array([[ 0.01  

In [6]:
import numpy as np

min_val = 1000
max_val = -1000
for x in dataset.tensor_val_data.as_numpy_iterator():
    seq = x[0]['modified_sequence']

    line_min = np.min(seq)
    if line_min < 0:
        print(x[0])
        # for s in seq:
        #     print(s)

    min_val = np.min([min_val, np.min(seq)])
    max_val = np.max([max_val, np.max(seq)])

print(f"min: {min_val}, max: {max_val}")

{'modified_sequence': array([[21, 16,  8, ...,  0,  0,  0],
       [56,  1,  1, ...,  0,  0,  0],
       [21, 15, 10, ...,  0,  0,  0],
       ...,
       [21,  4, 17, ...,  0,  0,  0],
       [21,  8, 10, ...,  0,  0,  0],
       [56,  5, 16, ...,  0,  0,  0]]), 'precursor_charge_onehot': array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32), 'collision_energy_aligned_normed': array([0.3706906 , 0.25769696, 0.22851454, 0.23789184, 0.35      ,
       0.51638985, 0.23172608, 0.40745035, 0.35      , 0.26870653,
       0.37040877, 0.2821617 , 0.38349447, 0.21373801, 0.35      ,
       0.32424766, 0.37311274, 0.31784788, 0.21831833, 0.23825108,
       0.24516802, 0.2479425 , 0.3194331 , 0.3254857 , 0.35      ,
       0.35      , 0.38135216, 0.35      , 0.3894306 , 0.35      ,
       0.30430368, 0.35      , 0.42888635, 0.

/nfs/home/students/j.poschenrieder/miniconda3/envs/dlomix/lib/python3.10/site-packages/datasets/utils/tf_utils.py:131: RuntimeWarning: invalid value encountered in cast
  array = array.astype(cast_dtype)


In [7]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
